<a href="https://colab.research.google.com/github/IkeLyons/HW4-Sequence-Models/blob/main/SequenceModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this homework I am going to try to generate text based off of the script of Star Wars (dataset link: https://www.kaggle.com/xvivancos/star-wars-movie-scripts). DESCRIBE HERE WHY THIS NEEDS TO BE SOLVED USING SEQUENCE MODELS!!!!!!!!!!!!!!!!! 

In [2]:
! pip install kaggle

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download xvivancos/star-wars-movie-scripts

  0% 0.00/158k [00:00<?, ?B/s]
100% 158k/158k [00:00<00:00, 62.5MB/s]


In [6]:
! unzip star-wars-movie-scripts.zip

Archive:  star-wars-movie-scripts.zip
  inflating: SW_EpisodeIV.txt        
  inflating: SW_EpisodeV.txt         
  inflating: SW_EpisodeVI.txt        
  inflating: wordcloud_masks/r2d2.png  
  inflating: wordcloud_masks/rebel alliance.png  
  inflating: wordcloud_masks/vader.png  
  inflating: wordcloud_masks/yoda.png  


We are using TensorFlow and some of the Keras utilties in order to solve this problem.

In [7]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np

In [8]:
text = open('SW_EpisodeIV.txt', 'rb').read().decode(encoding='utf-8')
text = text.lower()
text = text.replace('"', '')
print(text[:500])

character dialogue
1 threepio did you hear that?  they've shut down the main reactor.  we'll be destroyed for sure.  this is madness!
2 threepio we're doomed!
3 threepio there'll be no escape for the princess this time.
4 threepio what's that?
5 threepio i should have known better than to trust the logic of a half-sized thermocapsulary dehousing assister...
6 luke hurry up!  come with me!  what are you waiting for?!  get in gear!
7 threepio artoo! artoo-detoo, where are you?
8 threepio at last! 


We can see that the script has 3 main parts: the line number, the character speaking, and what the character said. It will be interesting to see how the models are able to recreate this.

In [9]:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '?', '\\', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


# Task 1

Now we need to transform this script data and character encoding, into the actual datasets that we are going to use for training.

In [24]:
import keras as K

seq_length = 50
X_train = []
y_train = []
# X_train needs to be an array of the indicies of the characters in the sequence
# y_train needs to be the correct next character
for i in range(0, len(text) - seq_length):
  X = text[i:i + seq_length]
  y = text[i + seq_length]
  X_train.append([chars.index(x) for x in X])
  y_train.append(chars.index(y))

X_train = np.reshape(X_train, (len(X_train), seq_length))
# convert to one hot encoding
y_train = K.utils.np_utils.to_categorical(y_train)

print(X_train[0])
print(y_train[0])
print(''.join([chars[x] for x in X_train[0]]))

[22 27 20 37 20 22 39 24 37  1 23 28 20 31 34 26 40 24  0  8  1 39 27 37
 24 24 35 28 34  1 23 28 23  1 44 34 40  1 27 24 20 37  1 39 27 20 39 18
  1  1]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
character dialogue
1 threepio did you hear that?  


Doing the Training, Validation, Test split

In [26]:
from sklearn.model_selection import train_test_split 

# train: 80%, val: 10%, test: 10%
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1) 

# Task 1 (Part 1)

Now that our dataset is ready, it is time to create the actual model